# 2021년도 코드로 TFT 실행해보기

In [1]:
# Import Module
import sys
import os
import argparse
import shutil
import random
from pathlib import Path
import datetime
from tqdm import tqdm

import pandas as pd
import numpy as np
import torch
import pytorch_lightning as pl

from pytorch_forecasting.data import (
    TimeSeriesDataSet,
    GroupNormalizer
)
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import (
    ModelCheckpoint,
    EarlyStopping,
    LearningRateMonitor
)
from pytorch_forecasting.metrics import SMAPE
from pytorch_forecasting.models import TemporalFusionTransformer

c:\Users\kijeong\anaconda3\envs\torch_book\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Fixed random seed
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42) # Seed 고정

In [3]:
cluster = {
    0: [19, 20, 21, 49, 50, 51],
    1: [1, 5, 9, 34],
    2: [4, 10, 11, 12, 28, 29, 30, 36, 40, 41, 42, 59, 60],
    3: [2, 3, 6, 7, 8, 13, 14, 15, 16, 17, 18, 22, 23, 24, 25, 26, 27, 31, 32, 33, 35, 37, 38, 39, 43, 44, 45, 46, 47, 48, 52, 53, 54, 55, 56, 57, 58],
}

In [54]:
num_epochs = 66
batch_size = 128
seed = 42
encoder_length_in_weeks = 5

params = {
    'gradient_clip_val': 0.9658579636307634,
    'hidden_size': 20,
    'dropout': 0.19610151695402608,
    'hidden_continuous_size': 10,
    'attention_head_size': 4,
    'learning_rate': 0.08
}

In [55]:
# learning rate determined by a cv run with train data less 1 trailing week as validation 
lrs = [0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306 , 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.005099279397234306, 0.005099279397234306, 0.005099279397234306, 0.005099279397234306,
       0.005099279397234306, 0.005099279397234306, 0.005099279397234306, 0.005099279397234306,
       0.005099279397234306, 0.0005099279397234307, 0.0005099279397234307, 0.0005099279397234307,
       0.0005099279397234307, 0.0005099279397234307, 0.0005099279397234307]

In [5]:
# Load Data
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
sample_submission = pd.read_csv('../data/sample_submission.csv')

In [6]:
# copy
train_df = train.copy()
test_df = test.copy()

In [7]:
# Drop the unnecessary columns for analysis
train_df.drop(columns=['num_date_time', '일조(hr)', '일사(MJ/m2)'], inplace=True)
test_df.drop(columns=['num_date_time'], inplace=True)

In [8]:
# Rename columns
train_df.columns = ['building_num', 'date', 'temperature', 'precipitation', 'windspeed', 'humidity', 'power_consumption']
test_df.columns = ['building_num', 'date', 'temperature', 'precipitation', 'windspeed', 'humidity']

## train data preprocessing

In [9]:
# Process the missing values of precipitation
train_df['precipitation'].fillna(0, inplace=True) # 강수량은 0으로 채움

In [10]:
# Process the missing values of wind speed and humidity
def fill_missing_with_avg(df, columns):
    for i in range(len(df)):
        if pd.isna(df.loc[i, columns]):
            
            prev_value_sum = df.loc[i-4:i-1, columns].sum()
            next_value_sum = df.loc[i+1:i+4, columns].sum()
            avg_value = (prev_value_sum + next_value_sum) / 8

            df.loc[i, columns] = avg_value

fill_missing_with_avg(train_df, 'windspeed')
fill_missing_with_avg(train_df, 'humidity')

In [11]:
# Convert the type of date from object to datetime
train_df['date'] = pd.to_datetime(train_df['date'], format='%Y%m%d %H')

In [12]:
# Decomposition of time elements
train_df['month'] = train_df.date.dt.month
train_df['day'] = train_df.date.dt.day
train_df['weekday'] = train_df.date.dt.weekday
train_df['hour'] = train_df.date.dt.hour

In [13]:
# Add time periodic columns
day_periodic = (train_df['day'] - 1) / 30 # 1부터 시작하므로 1을 빼줌
weekday_periodic = train_df['weekday'] / 6
hour_periodic = train_df['hour'] / 23

def sin_transform(values):
    return np.sin(2 * np.pi * values)
def cos_transform(values):
    return np.cos(2 * np.pi * values)

train_df['sin_weekday'] = sin_transform(weekday_periodic)
train_df['cos_weekday'] = cos_transform(weekday_periodic)
train_df['sin_hour'] = sin_transform(hour_periodic)
train_df['cos_hour'] = cos_transform(hour_periodic)

In [14]:
month_dummy = pd.get_dummies(train_df['month']).rename(columns={6:'month_6', 7:'month_7', 8:'month_8'})
train_df = pd.concat([train_df, month_dummy[['month_6', 'month_7']]], axis=1)

In [15]:
# Add holiday column
# holiday = 1, workday = 0
train_df['holiday'] = train_df.apply(lambda x : 0 if x['day']<5 else 1, axis = 1)
train_df.loc[(train_df.date == datetime.date(2022, 6, 6))&(train_df.date == datetime.date(2022, 8, 15)), 'holiday'] = 1

In [16]:
# Add DI column (DI : Discomfort Index (불쾌지수))
train_df['DI'] = 9/5*train_df['temperature'] - 0.55*(1 - train_df['humidity']/100) * (9/5*train_df['humidity'] - 26) + 32

In [17]:
# Add CDH column (CDH : Cooling Degree-Day)
# CDD(냉방도일) -> CDH(냉방도시)로 데이터에 맞게 변형한 개념
# 냉방도일 : 어느 지방의 더운 정도를 나타내는 지표로 사용 (sum(해당 일의 기온 - 기준온도))
# 냉방도일이 크면 연료의 소비량이 많아짐
# 실내온도가 같아도 외부 기온은 지역별로 다름 -> 지역마다 값이 다르게 나타남 -> 이 데이터에서는 건물별로 다르다고 얘기할 수 있음
# 냉방도시 : sum(해당 시간의 기온 - 기준온도)
def CDH(df, num_building):
    df_ = df.copy()
    cdhs = np.array([])
    for num in range(1, num_building+1, 1):
        cdh = []
        cdh_df = df_[df_['building_num'] == num_building]
        cdh_temp = cdh_df['temperature'].values # Series로도 돌릴 수 있지만 array로 돌리는게 속도가 훨씬 빠름
        for i in range(len(cdh_temp)):
            if i < 11:
                cdh.append(np.sum(cdh_temp[:(i+1)] - 26))
            else:
                cdh.append(np.sum(cdh_temp[(i-11):(i+1)] - 26))
        
        cdhs = np.concatenate([cdhs, cdh])
    
    return cdhs

train_df['CDH'] = CDH(train_df, 100)

In [18]:
# 건물별, 요일별, 시간별 발전량 평균 넣어주기
power_mean = pd.pivot_table(train_df, values = 'power_consumption', index = ['building_num', 'hour', 'weekday'], aggfunc = np.mean).reset_index()
tqdm.pandas()
train_df['day_hour_mean'] = train_df.progress_apply(lambda x : power_mean.loc[(power_mean.building_num == x['building_num']) & (power_mean.hour == x['hour']) & (power_mean.weekday == x['weekday']), 'power_consumption'].values[0], axis = 1)

100%|██████████| 204000/204000 [02:09<00:00, 1571.17it/s]


In [19]:
# 건물별, 요일별, 시간별 발전량 표준편차 넣어주기
power_std = pd.pivot_table(train_df, values = 'power_consumption', index = ['building_num', 'hour', 'weekday'], aggfunc = np.std).reset_index()
tqdm.pandas()
train_df['day_hour_std'] = train_df.progress_apply(lambda x : power_std.loc[(power_std.building_num == x['building_num']) & (power_std.hour == x['hour']) & (power_std.weekday == x['weekday']), 'power_consumption'].values[0], axis = 1)

100%|██████████| 204000/204000 [01:44<00:00, 1957.30it/s]


In [20]:
# 건물별 시간별 발전량 평균 넣어주기
power_hour_mean = pd.pivot_table(train_df, values = 'power_consumption', index = ['building_num', 'hour'], aggfunc = np.mean).reset_index()
tqdm.pandas()
train_df['hour_mean'] = train_df.progress_apply(lambda x : power_hour_mean.loc[(power_hour_mean.building_num == x['building_num']) & (power_hour_mean.hour == x['hour']) ,'power_consumption'].values[0], axis = 1)

100%|██████████| 204000/204000 [00:51<00:00, 3979.28it/s]


In [21]:
# 건물별 시간별 발전량 표준편차 넣어주기
power_hour_std = pd.pivot_table(train_df, values = 'power_consumption', index = ['building_num', 'hour'], aggfunc = np.std).reset_index()
tqdm.pandas()
train_df['hour_std'] = train_df.progress_apply(lambda x : power_hour_std.loc[(power_hour_std.building_num == x['building_num']) & (power_hour_std.hour == x['hour']) ,'power_consumption'].values[0], axis = 1)

100%|██████████| 204000/204000 [00:48<00:00, 4224.85it/s]


In [22]:
train_df2 = train_df.copy()

In [23]:
train_df2.drop(columns=['month', 'day', 'weekday', 'hour'], inplace=True)

In [24]:
train_df2['time_idx'] = (train_df2.loc[:, 'date'] - train_df2.iloc[0, 1]).astype('timedelta64[h]').astype('int')

In [25]:
train_df2.head()

,building_num,date,temperature,precipitation,windspeed,humidity,power_consumption,sin_weekday,cos_weekday,sin_hour,...,month_6,month_7,holiday,DI,CDH,day_hour_mean,day_hour_std,hour_mean,hour_std,time_idx
0,1,2022-06-01 00:00:00,18.6,0.0,0.9,42.0,1085.28,0.866025,-0.5,0.000000,...,1,0,0,49.6576,-11.0,1774.744615,517.982222,1706.318118,446.882767,0
1,1,2022-06-01 01:00:00,18.0,0.0,1.1,45.0,1047.36,0.866025,-0.5,0.269797,...,1,0,0,47.7625,-22.8,1687.347692,500.769931,1622.620235,439.662704,1
2,1,2022-06-01 02:00:00,17.7,0.0,1.5,45.0,974.88,0.866025,-0.5,0.519584,...,1,0,0,47.2225,-35.3,1571.483077,465.227458,1506.971294,412.071906,2
3,1,2022-06-01 03:00:00,16.7,0.0,1.4,48.0,953.76,0.866025,-0.5,0.730836,...,1,0,0,44.7856,-47.9,1522.153846,436.601091,1437.365647,391.205981,3
4,1,2022-06-01 04:00:00,18.4,0.0,2.8,43.0,986.40,0.866025,-0.5,0.887885,...,1,0,0,49.0061,-60.1,1506.793846,405.518091,1447.321412,381.099697,4


## test data preprocessing

In [26]:
# Process the missing values of precipitation
test_df['precipitation'].fillna(0, inplace=True) # 강수량은 0으로 채움

In [27]:
# Process the missing values of wind speed and humidity
def fill_missing_with_avg(df, columns):
    for i in range(len(df)):
        if pd.isna(df.loc[i, columns]):
            
            prev_value_sum = df.loc[i-4:i-1, columns].sum()
            next_value_sum = df.loc[i+1:i+4, columns].sum()
            avg_value = (prev_value_sum + next_value_sum) / 8

            df.loc[i, columns] = avg_value

fill_missing_with_avg(test_df, 'windspeed')
fill_missing_with_avg(test_df, 'humidity')

In [28]:
# Convert the type of date from object to datetime
test_df['date'] = pd.to_datetime(test_df['date'], format='%Y%m%d %H')

In [29]:
# Decomposition of time elements
test_df['month'] = test_df.date.dt.month
test_df['day'] = test_df.date.dt.day
test_df['weekday'] = test_df.date.dt.weekday
test_df['hour'] = test_df.date.dt.hour

In [30]:
# Add time periodic columns
day_periodic = (test_df['day'] - 1) / 30 # 1부터 시작하므로 1을 빼줌
weekday_periodic = test_df['weekday'] / 6
hour_periodic = test_df['hour'] / 23

def sin_transform(values):
    return np.sin(2 * np.pi * values)
def cos_transform(values):
    return np.cos(2 * np.pi * values)

test_df['sin_weekday'] = sin_transform(weekday_periodic)
test_df['cos_weekday'] = cos_transform(weekday_periodic)
test_df['sin_hour'] = sin_transform(hour_periodic)
test_df['cos_hour'] = cos_transform(hour_periodic)

In [31]:
test_df['month_6'] = 0
test_df['month_7'] = 0

In [32]:
# Add holiday column
# holiday = 1, workday = 0
test_df['holiday'] = test_df.apply(lambda x : 0 if x['day']<5 else 1, axis = 1)
test_df.loc[(test_df.date == datetime.date(2022, 6, 6))&(test_df.date == datetime.date(2022, 8, 15)), 'holiday'] = 1

In [33]:
# Add DI column (DI : Discomfort Index (불쾌지수))
test_df['DI'] = 9/5*test_df['temperature'] - 0.55*(1 - test_df['humidity']/100) * (9/5*test_df['humidity'] - 26) + 32

In [34]:
# Add CDH column (CDH : Cooling Degree-Day)
# CDD(냉방도일) -> CDH(냉방도시)로 데이터에 맞게 변형한 개념
# 냉방도일 : 어느 지방의 더운 정도를 나타내는 지표로 사용 (sum(해당 일의 기온 - 기준온도))
# 냉방도일이 크면 연료의 소비량이 많아짐
# 실내온도가 같아도 외부 기온은 지역별로 다름 -> 지역마다 값이 다르게 나타남 -> 이 데이터에서는 건물별로 다르다고 얘기할 수 있음
# 냉방도시 : sum(해당 시간의 기온 - 기준온도)
def CDH(df, num_building):
    df_ = df.copy()
    cdhs = np.array([])
    for num in range(1, num_building+1, 1):
        cdh = []
        cdh_df = df_[df_['building_num'] == num_building]
        cdh_temp = cdh_df['temperature'].values # Series로도 돌릴 수 있지만 array로 돌리는게 속도가 훨씬 빠름
        for i in range(len(cdh_temp)):
            if i < 11:
                cdh.append(np.sum(cdh_temp[:(i+1)] - 26))
            else:
                cdh.append(np.sum(cdh_temp[(i-11):(i+1)] - 26))
        
        cdhs = np.concatenate([cdhs, cdh])
    
    return cdhs

test_df['CDH'] = CDH(test_df, 100)

In [35]:
# 건물별, 요일별, 시간별 발전량 평균 넣어주기
tqdm.pandas()
test_df['day_hour_mean'] = test_df.progress_apply(lambda x : power_mean.loc[(power_mean.building_num == x['building_num']) & (power_mean.hour == x['hour']) & (power_mean.weekday == x['weekday']), 'power_consumption'].values[0], axis = 1)

100%|██████████| 16800/16800 [00:06<00:00, 2585.54it/s]


In [36]:
# 건물별, 요일별, 시간별 발전량 표준편차 넣어주기
tqdm.pandas()
test_df['day_hour_std'] = test_df.progress_apply(lambda x : power_std.loc[(power_std.building_num == x['building_num']) & (power_std.hour == x['hour']) & (power_std.weekday == x['weekday']), 'power_consumption'].values[0], axis = 1)

100%|██████████| 16800/16800 [00:06<00:00, 2446.47it/s]


In [37]:
# 건물별 시간별 발전량 평균 넣어주기
tqdm.pandas()
test_df['hour_mean'] = test_df.progress_apply(lambda x : power_hour_mean.loc[(power_hour_mean.building_num == x['building_num']) & (power_hour_mean.hour == x['hour']) ,'power_consumption'].values[0], axis = 1)

100%|██████████| 16800/16800 [00:04<00:00, 4080.49it/s]


In [38]:
# 건물별 시간별 발전량 표준편차 넣어주기
tqdm.pandas()
test_df['hour_std'] = test_df.progress_apply(lambda x : power_hour_std.loc[(power_hour_std.building_num == x['building_num']) & (power_hour_std.hour == x['hour']) ,'power_consumption'].values[0], axis = 1)

100%|██████████| 16800/16800 [00:04<00:00, 3867.29it/s]


In [39]:
test_df2 = test_df.copy()

In [40]:
test_df2.drop(columns=['month', 'day', 'weekday', 'hour'], inplace=True)

In [41]:
test_df2['time_idx'] = (test_df2.loc[:, 'date'] - train_df2.iloc[0, 1]).astype('timedelta64[h]').astype('int')

In [42]:
test_df2.head()

,building_num,date,temperature,precipitation,windspeed,humidity,sin_weekday,cos_weekday,sin_hour,cos_hour,month_6,month_7,holiday,DI,CDH,day_hour_mean,day_hour_std,hour_mean,hour_std,time_idx
0,1,2022-08-25 00:00:00,23.5,0.0,2.2,72,1.224647e-16,-1.0,0.000000,1.000000,0,0,1,58.3456,-5.8,1627.80,446.984913,1706.318118,446.882767,2040
1,1,2022-08-25 01:00:00,23.0,0.0,0.9,72,1.224647e-16,-1.0,0.269797,0.962917,0,0,1,57.4456,-12.1,1550.08,449.091398,1622.620235,439.662704,2041
2,1,2022-08-25 02:00:00,22.7,0.0,1.5,75,1.224647e-16,-1.0,0.519584,0.854419,0,0,1,57.8725,-18.7,1431.12,415.453568,1506.971294,412.071906,2042
3,1,2022-08-25 03:00:00,22.1,0.0,1.3,78,1.224647e-16,-1.0,0.730836,0.682553,0,0,1,57.9376,-25.5,1372.20,378.117772,1437.365647,391.205981,2043
4,1,2022-08-25 04:00:00,21.8,0.0,1.0,77,1.224647e-16,-1.0,0.887885,0.460065,0,0,1,56.9961,-32.5,1381.72,360.348483,1447.321412,381.099697,2044


## transform the dataset for TFT

In [43]:
categorical = ['building_num', 'month_6', 'month_7', 'holiday']

for col in categorical:
        train_df2[col] = train_df2[col].astype(str).astype('category')

for col in categorical:
        test_df2[col] = test_df2[col].astype(str).astype('category')

In [44]:
test_df2['power_consumption'] = np.nan
test_df2 = test_df2[test_df2.columns[0:6].to_list() + test_df2.columns[-1:].to_list() + test_df2.columns[7:-1].to_list()]

In [45]:
test_df2.columns

Index(['building_num', 'date', 'temperature', 'precipitation', 'windspeed',
       'humidity', 'power_consumption', 'cos_weekday', 'sin_hour', 'cos_hour',
       'month_6', 'month_7', 'holiday', 'DI', 'CDH', 'day_hour_mean',
       'day_hour_std', 'hour_mean', 'hour_std', 'time_idx'],
      dtype='object')

In [46]:
test_df2['day_hour_mean'] = np.nan
test_df2['day_hour_std'] = np.nan
test_df2['hour_mean'] = np.nan
test_df2['hour_std'] = np.nan

In [47]:
def load_dataset(train_df, validate=False):
    max_encoder_length = 24 * 7 * encoder_length_in_weeks
    max_prediction_length = 24 * 7
    training_cutoff = train_df['time_idx'].max() - max_prediction_length # 2039 - 24*7 = 1871

    tr_ds = TimeSeriesDataSet(
      train_df[lambda x: x.time_idx <=training_cutoff] if validate else train_df, 
      time_idx = "time_idx",
      target = "power_consumption",
      group_ids=["building_num"],
      min_encoder_length = 1,
      max_encoder_length = max_encoder_length, 
      min_prediction_length=1, 
      max_prediction_length=max_prediction_length,

      # Known Inputs 알고 있는 변수
      time_varying_known_categoricals = categorical, 
      time_varying_known_reals=[
            "time_idx",
            "temperature",
            "windspeed",
            "humidity",
            "precipitation",
            'sin_weekday',
            'cos_weekday',
            'sin_hour', 
            'cos_hour',
            'DI',
            'CDH'
        ],
      target_normalizer=GroupNormalizer(groups=["building_num"], transformation="softplus"),
      
      # Future Inputs
      time_varying_unknown_categoricals=[],
      time_varying_unknown_reals=[
            "power_consumption",
            "day_hour_mean",
            "day_hour_std",
            "hour_mean",
            "hour_std",
        ],

        
        add_relative_time_idx=True,  # add as feature
        add_target_scales=True,  # add as feature
        add_encoder_length=True,  # add as feature
    )
  

    va_ds = None
    if validate:
        va_ds = TimeSeriesDataSet.from_dataset(
        tr_ds, train_df, predict=True, stop_randomization=True
    )

    return tr_ds, va_ds

In [48]:
tr_ds, va_ds = load_dataset(train_df2, validate=False)

## modeling

In [57]:
# # training
# def fit(seed, tr_ds, va_loader=None):
#     seed_everything(seed)

#     # create dataloaders for model
#     tr_loader = tr_ds.to_dataloader(
#         train=True, batch_size=batch_size, num_workers=12
#     )

#     if va_loader is not None:
#         # stop training, when loss metric does not improve on validation set
#         early_stopping_callback = EarlyStopping(
#             monitor="val_loss",
#             min_delta=1e-4,
#             patience=20,
#             verbose=True,
#             mode="min"
#         )
#         lr_logger = LearningRateMonitor(logging_interval="epoch")  # log the learning rate
#         callbacks = [lr_logger, early_stopping_callback]
#     else:
#         # gather 10 checkpoints with best traing loss
#         checkpoint_callback = ModelCheckpoint(
#             monitor='train_loss',
#             dirpath='tft/',
#             filename=f'seed={seed}'+'-{epoch:03d}-{train_loss:.2f}',
#             save_top_k=10
#         )
#         callbacks = [checkpoint_callback]

#     # create trainer
#     trainer = pl.Trainer(
#         max_epochs=66,
#         gpus=-1,
#         gradient_clip_val=params['gradient_clip_val'],
#         limit_train_batches=30,
#         callbacks=callbacks,
#         logger=TensorBoardLogger('tft_logs', name='my_model')
#     )

#     # use pre-deterined leraning rate schedule for final submission
#     learning_rate = lrs if va_loader is None else params['learning_rate']

#     # initialise model with pre-determined hyperparameters
#     tft = TemporalFusionTransformer.from_dataset(
#         tr_ds,
#         learning_rate=learning_rate,
#         hidden_size=params['hidden_size'],
#         attention_head_size=params['attention_head_size'],
#         dropout=params['dropout'],
#         hidden_continuous_size=params['hidden_continuous_size'],
#         output_size=1,
#         loss=SMAPE(), # SMAPE loss
#         log_interval=10,  # log example every 10 batches
#         logging_metrics=[SMAPE()],
#         reduce_on_plateau_patience=4  # reduce learning automatically
#     )
#     print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

#     kwargs = {'train_dataloader': tr_loader}
#     if va_loader:
#         kwargs['val_dataloaders'] = va_loader

#     # fit network
#     trainer.fit(
#         tft,
#         **kwargs
#     )

#     best_model_path = trainer.checkpoint_callback.best_model_path
#     print(f'best_model_path={best_model_path}')
#     best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

#     return best_tft

In [60]:
seed_everything(seed)

va_loader = None

# create dataloaders for model
tr_loader = tr_ds.to_dataloader(
    train=True, batch_size=batch_size, num_workers=6
)

if va_loader is not None:
    # stop training, when loss metric does not improve on validation set
    early_stopping_callback = EarlyStopping(
        monitor="val_loss",
        min_delta=1e-4,
        patience=20,
        verbose=True,
        mode="min"
    )
    lr_logger = LearningRateMonitor(logging_interval="epoch")  # log the learning rate
    callbacks = [lr_logger, early_stopping_callback]
else:
    # gather 10 checkpoints with best traing loss
    checkpoint_callback = ModelCheckpoint(
        monitor='train_loss',
        dirpath='tft/',
        filename=f'seed={seed}'+'-{epoch:03d}-{train_loss:.2f}',
        save_top_k=10
    )
    callbacks = [checkpoint_callback]

C:\Users\kijeong\AppData\Roaming\Python\Python39\site-packages\torch\utils\data\dataloader.py:487: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 8 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [71]:
# create trainer
trainer = pl.Trainer(
    max_epochs=10,
    accelerator='gpu'
    devices=-1
    gradient_clip_val=params['gradient_clip_val'],
    limit_train_batches=30,
    callbacks=callbacks,
    logger=TensorBoardLogger('tft_logs', name='my_model')
)

C:\Users\kijeong\AppData\Roaming\Python\Python39\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:478: LightningDeprecationWarning: Setting `Trainer(gpus=-1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=-1)` instead.
  rank_zero_deprecation(
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [72]:
# use pre-deterined leraning rate schedule for final submission
learning_rate = lrs if va_loader is None else params['learning_rate']

In [73]:
# initialise model with pre-determined hyperparameters
tft = TemporalFusionTransformer.from_dataset(
    tr_ds,
    learning_rate=learning_rate,
    hidden_size=params['hidden_size'],
    attention_head_size=params['attention_head_size'],
    dropout=params['dropout'],
    hidden_continuous_size=params['hidden_continuous_size'],
    output_size=1,
    loss=SMAPE(), # SMAPE loss
    log_interval=10,  # log example every 10 batches
    logging_metrics=[SMAPE()],
    reduce_on_plateau_patience=4  # reduce learning automatically
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

Number of parameters in network: 57.7k


C:\Users\kijeong\AppData\Roaming\Python\Python39\site-packages\pytorch_lightning\utilities\parsing.py:269: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(


In [74]:
kwargs = {'train_dataloader': tr_loader}
if va_loader:
    kwargs['val_dataloaders'] = va_loader

In [80]:
# fit network
trainer.fit(
    tft,
    kwargs
)

best_model_path = trainer.checkpoint_callback.best_model_path
print(f'best_model_path={best_model_path}')
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)
# 20분 돌려도 안 돌아감..

C:\Users\kijeong\AppData\Roaming\Python\Python39\site-packages\pytorch_lightning\trainer\configuration_validator.py:108: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | SMAPE                           | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 2.0 K 
3  | prescalers                         | ModuleDict                      | 400   
4  | static_variable_selection          | VariableSelectionNetwork        | 2.5 K 
5  | encoder_variable_selection         | VariableSelectionNetwork        | 19.2 K
6  | decoder_variable_selection         | Vari

Epoch 0:   0%|          | 0/30 [00:00<?, ?it/s] 

C:\Users\kijeong\AppData\Roaming\Python\Python39\site-packages\torch\utils\data\dataloader.py:487: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 8 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
# predict 1 week
def forecast(ckpt, train_df, test_df):
    # load model
    best_tft = TemporalFusionTransformer.load_from_checkpoint(ckpt)
    max_encoder_length = best_tft.dataset_parameters['max_encoder_length']
    max_prediction_length = best_tft.dataset_parameters['max_prediction_length']

    assert max_encoder_length == 5*24*7 and max_prediction_length == 1*24*7

    # use 5 weeks of training data at the end
    encoder_data = train_df[lambda x: x.time_idx > x.time_idx.max() - max_encoder_length]

    # get last entry from training data
    last_data = train_df.iloc[[-1]]

    # fill NA target value in test data with last values from the train dataset
    target_cols = [c for c in test_df.columns if 'target' in c]
    for c in target_cols:
        test_df.loc[:, c] = last_data[c].item()

    decoder_data = test_df

    # combine encoder and decoder data. decoder data is to be predicted
    new_prediction_data = pd.concat([encoder_data, decoder_data], ignore_index=True)
    new_raw_predictions, new_x = best_tft.predict(new_prediction_data, mode="raw", return_x=True)

    # num_labels: mapping from 'num' categorical feature to index in new_raw_predictions['prediction']
    #             {'5': 4, '6': 6, ...}
    # new_raw_predictions['prediction'].shape = (60, 168, 1)
    num_labels = best_tft.dataset_parameters['categorical_encoders']['num'].classes_

    preds = new_raw_predictions['prediction'].squeeze()

    sub_df = pd.read_csv(../data/sample_submission.csv")

    # get prediction for each building (num)
    for n, ix in num_labels.items():
        sub_df.loc[sub_df.num_date_time.str.startswith(f"{n} "), 'answer'] = preds[ix].numpy()

    # save predction to a csv file
    outfn = CSVROOT+'/'+(Path(ckpt).stem + '.csv')
    print(outfn)
    sub_df.to_csv(outfn, index=False)

In [ ]:
def ensemble(outfn):
    # get all prediction csv files
    fns = list(glob.glob(CSVROOT+"/*.csv"))
    df0 = pd.read_csv(fns[0])
    df = pd.concat([df0] + [pd.read_csv(fn).loc[:,'answer'] for fn in fns[1:]], axis=1)
    # get median of all predcitions
    df['median'] = df.iloc[:,1:].median(axis=1)
    df = df[['num_date_time', 'median']]
    df = df.rename({'median': 'answer'}, axis=1)
    # save to submission file
    df.to_csv(outfn, index=False)

In [ ]:
import glob
print("### FORECAST ###")
for p in glob.glob(CKPTROOT + "/*.ckpt"):
    forecast(p, train_df, test_df)

In [ ]:
print("### ENSEMBLING ###")
ensemble(SUBFN)